In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install d2l
!pip install optuna
%cd /content/drive/MyDrive/EE-519/MP1

import numpy as np
import math
import torch
import optuna
import torchvision
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader, Dataset
from d2l import torch as d2l
from optuna.trial import TrialState

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.6/112.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
  

In [2]:
def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 2, 4)
    layers = []

    in_features = 16
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, CLASSES))
    #layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)

In [9]:
def objective(trial):
    # Generate the model.
    model = define_model(trial).to(DEVICE)
    model = model.type(torch.float64)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    criterion = nn.MSELoss()

    # Get the dataset.
    #train_loader, valid_loader, mu, nComp, pca_components, saved_labels, num_epochs = get_data()

    # Training of the model.
    for epoch in range(num_epochs):

        for i, sample in enumerate(train_loader):
            inputs, labels = sample[:,:-1], sample[:,-1]
            inputs = inputs.type(torch.float64)
            labels = labels.type(torch.float64)
            #zero gradients here instead??
            #forward pass
            outputs = model(inputs)
            #loss
            labels = labels.type(torch.float64)
            outputs = outputs.type(torch.float64)
            loss = criterion(outputs, labels)
            #zero gradients
            optimizer.zero_grad()
            #backward pass
            loss.mean().backward()
            optimizer.step()


        # Validation of the model.
        model.eval()
        correct = 0
        total = 0

        #tensor to store outputs
        FINAL_OUTPUTS = torch.zeros(0, dtype=torch.float32)

        with torch.no_grad():
            #for sample in valid_loader:
            inputs = valid_data[:,:-1]#, valid_data[:,-1]
            FINAL_OUTPUTS = torch.cat((FINAL_OUTPUTS, model(inputs)), 0)
            #labels = labels.reshape(FINAL_OUTPUTS.size())

        #set val_data = to valid_data
        outputs_data = valid_data
        #reshape outputs to put into val_data
        outputs_data[:, -1] = torch.zeros([611,], dtype=torch.float32)
        outputs_data[:, -1] = FINAL_OUTPUTS.reshape([611,])
        outputs_data = np.dot(pca.transform(outputs_data)[:, :nComp], pca_components[:nComp, :])
        outputs = outputs_data[:,-1]

        labels = saved_labels.to_numpy()

        def destandardize (Y, mean, std):
            input_destdize = (Y * std) + mean
            return input_destdize

        labels = destandardize(labels, denorm_mean, denorm_std)
        outputs = destandardize(outputs, denorm_mean, denorm_std)

        for i in range(len(outputs)):
            if outputs[i] <= 25:
                outputs[i] = 0
            elif outputs[i] <= 50 and outputs[i] > 25:
                outputs[i] = 1
            elif outputs[i] <= 200 and outputs[i] > 50:
                outputs[i] = 2
            else:
                outputs[i] = 3
            if labels[i] <= 25:
                labels[i] = 0
            elif labels[i] <= 50 and labels[i] > 25:
                labels[i] = 1
            elif labels[i] <= 200 and labels[i] > 50:
                labels[i] = 2
            else:
                labels[i] = 3

        finals = pd.DataFrame()
        finals['outputs'] = outputs
        finals['labels'] = labels
        
        total = len(outputs)
        correct = (outputs == labels).sum().item()
        accuracy = correct / total
        
        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        return accuracy


In [4]:
DEVICE = torch.device('cpu')
num_epochs = 3200
batch_size = 350
num_workers = 0
CLASSES = 1

#def get_data():

dataset = pd.read_csv('data/heatflow_resid_train.csv')

train_data = dataset[['hf22_tgwt','cond_sur','cond_lcr','cond_man','cond_mcr','cond_uma','geop_mag','geop_grv','geop_dtb','eqi200n5','eqd200n5','geod_shr', 'geod_dil','geod_2nd','faultAl','vent_ing', 'hfqc_resid']]


#convert into test and validation sets
valid_data = train_data.iloc[2500:-1, :]
train_data = train_data.iloc[0:2500, :]

#normalize data
mean = train_data.mean()
std = train_data.std() 
denorm_mean = mean['hfqc_resid']
denorm_std = std['hfqc_resid']

train_data = (train_data - mean) / std
np.reshape(train_data, [2500,17])
valid_data = (valid_data - mean) / std
np.reshape(valid_data, [611,17])

saved_labels = valid_data['hfqc_resid']

#Apply PCA
components = 10#our number of features we want to reduce to
pca = PCA(n_components=components)
pca.fit(train_data)
train_features = pd.DataFrame(pca.components_)
#use a for loop to replace all the values in the dataframe with the new values
for i in range(components):
      train_data.iloc[i,:] = train_features.iloc[i,:]


#Apply PCA to validation set
components = 10#our number of features we want to reduce to
pca = PCA(n_components=components)
pca.fit(valid_data)
pca_components = pca.components_
valid_features = pd.DataFrame(pca.components_)
#use a for loop to replace all the values in the dataframe with the new values
for i in range(components):
    valid_data.iloc[i,:] = valid_features.iloc[i,:]

mu = np.mean(valid_data)
mu = mu['hfqc_resid']
nComp = 10

#convert datasets to tensors
tensored_data = torch.tensor(train_data.values, dtype=torch.float32)
valid_data = torch.tensor(valid_data.values, dtype=torch.float32)



valid_data = valid_data.type(torch.float64)
tensored_data = tensored_data.type(torch.float64)
#load data into dataloader
train_loader = torch.utils.data.DataLoader(dataset=tensored_data, batch_size=batch_size, shuffle=True)#, num_workers=num_workers
valid_loader = torch.utils.data.DataLoader(dataset=valid_data, batch_size=batch_size, shuffle=True)#, num_workers=num_workers

#return train_loader, valid_loader, mu, nComp, pca_components, saved_labels, num_epochs


In [10]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=600)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-04-25 07:44:21,089] A new study created in memory with name: no-name-7f769b65-69ac-4bda-8473-fcf74f36204c
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([350])) that is different to the input size (torch.Size([350, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([50])) that is different to the input size (torch.Size([50, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
[I 2023-04-25 07:44:21,224] Tri

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  81
  Number of complete trials:  19
Best trial:
  Value:  0.40425531914893614
  Params: 
    n_layers: 2
    n_units_l0: 98
    dropout_l0: 0.2953487309500333
    n_units_l1: 4
    dropout_l1: 0.45917796114217546
    optimizer: SGD
    lr: 0.0018400482656233165
